In [1]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


model_id = "google/flan-t5-base"
path = f'./data/{model_id}/base_model'



pipe = pipeline( "text2text-generation", model=model_id ,max_new_tokens = 100, model_kwargs={ "cache_dir" : path, 'device_map' : 'auto' })

hf = HuggingFacePipeline(model_id=model_id,pipeline = pipe)

hf.pipeline.model._validate_model_kwargs = lambda self: None



/home/adil/repo/idriss/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "the capital of morroc is rabat give me a city of Morocco ?"

print(chain.invoke({"question": question}))

Morocco is a country in the Middle East. Rabat is the capital of Morocco. The answer: Rabat.


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_community.document_loaders import SeleniumURLLoader
# loader = PyPDFLoader("./cv.pdf")
# data = loader.load_and_split()

# loader1 = TextLoader("./data.txt")
# data1 = loader1.load_and_split()
# data.extend(data1)

urls = [
    "https://telquel.ma/instant-t/2023/07/28/le-ministere-de-lenseignement-superieur-lance-a-la-rentree-ses-ecoles-de-codages-code-212_1823860/",
    "https://leseco.ma/maroc/code-212-le-premier-centre-ouvrira-a-agadir.html",
    "https://fr.wikipedia.org/wiki/Maroc"
]


loader = SeleniumURLLoader(urls=urls)

data = loader.load_and_split()

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

In [5]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(data, embeddings)

KeyboardInterrupt: 

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(hf, retriever=retriever )

In [ ]:
user_input = "c quoi code212"

result = chain({"query": user_input})

response = result["result"]

print("Chatbot:", response)

Chatbot: : the first centre ouvrira à Agadir LesEco.ma 27/09/2023 The first centre Code 212 ouvrira ses portes dès le mois d'octobre à l'Université Ibn Zohr d'Agadir. C'est ce qu'Abdellatif Miraoui, Minister de l'Enseignement supérieur, de la recherche scientifique e
